In [ ]:
## 🔑 Configuração da API OpenRouter


In [ ]:
# Verificar GPU disponível
!nvidia-smi

# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navegar para diretório de trabalho
%cd /content


In [ ]:
# Clone do repositório ValeTTS atualizado
!git clone https://github.com/wallaceblaia/ValeTTS-Colab.git ValeTTS

# 🔧 CORREÇÃO AUTOMÁTICA DE DIRETÓRIO E ESTRUTURA
import os

# Verificar se foi clonado em diretório aninhado
current_dir = os.getcwd()
print(f"📁 Diretório atual: {current_dir}")

if current_dir.endswith('/ValeTTS/ValeTTS'):
    # Corrigir diretório aninhado
    print("🔧 Detectado diretório aninhado, corrigindo...")
    os.chdir('/content/ValeTTS')
    print(f"📁 Movido para: {os.getcwd()}")
elif not current_dir.endswith('/ValeTTS'):
    # Navegar para o diretório correto
    os.chdir('/content/ValeTTS')
    print(f"📁 Navegado para: {os.getcwd()}")

# Criar diretórios essenciais
essential_dirs = ['logs', 'outputs', 'samples', 'checkpoints']
for dir_name in essential_dirs:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
        print(f"📂 Criado diretório: {dir_name}")

# Verificar estrutura do projeto
print("\n📁 Estrutura do projeto:")
!ls -la

# Verificar se os arquivos necessários existem
print("\n🔍 Verificando arquivos essenciais:")
essential_files = ['requirements.txt', 'setup.py', 'valetts/__init__.py', 'scripts/train_vits2.py']
for file in essential_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file}")

print(f"📁 Diretório final: {os.getcwd()}")

# Configurar PYTHONPATH corretamente
import sys
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"🐍 PYTHONPATH configurado: {current_dir}")


In [ ]:
# 📦 INSTALAÇÃO COMPLETA DE DEPENDÊNCIAS (INGLÊS + PORTUGUÊS)
import os
print(f"📁 Diretório atual: {os.getcwd()}")

# Dependências core do projeto
if os.path.exists('requirements.txt'):
    print("📦 Instalando dependências do projeto...")
    !pip install -r requirements.txt
else:
    print("⚠️  requirements.txt não encontrado, instalando dependências básicas...")
    !pip install torch torchaudio pytorch-lightning tensorboard pyyaml matplotlib scipy librosa

# 🌍 DEPENDÊNCIAS ESPECÍFICAS PARA PROCESSAMENTO DE INGLÊS
print("\n🌍 Instalando suporte específico para inglês...")

# Phonemizer para conversão texto-para-phonemes em inglês
print("🔤 Instalando phonemizer...")
!pip install phonemizer --quiet

# NLTK para processamento avançado de texto
print("📝 Instalando NLTK...")
!pip install nltk --quiet

# eSpeak backend para phonemizer (necessário para inglês)
print("🔊 Configurando eSpeak para phonemes ingleses...")
!sudo apt-get update -qq
!sudo apt-get install espeak espeak-data -y -qq

# Lightning para treinamento (versão mais recente)
print("⚡ Atualizando PyTorch Lightning...")
!pip install lightning --quiet

print("\n🧪 Testando phonemizer...")
try:
    from phonemizer import phonemize
    test_phonemes = phonemize("hello world", language='en-us', backend='espeak')
    print(f"✅ Phonemizer funcionando: '{test_phonemes}'")
except Exception as e:
    print(f"⚠️  Erro no phonemizer: {e}")

# Instalar o projeto em modo desenvolvimento
if os.path.exists('setup.py'):
    print("\n🔧 Instalando projeto em modo desenvolvimento...")
    !pip install -e .
else:
    print("\n⚠️  setup.py não encontrado, configurando PYTHONPATH...")
    import sys
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())

print("\n✅ TODAS AS DEPENDÊNCIAS INSTALADAS!")
print("   🔤 Phonemizer: Conversão G2P para inglês")
print("   📝 NLTK: Processamento de texto avançado")
print("   🔊 eSpeak: Backend para phonemes ingleses")
print("   ⚡ Lightning: Treinamento otimizado")


In [ ]:
# Criar diretório de dados
!mkdir -p data/generated

# Extrair dataset do Google Drive
import os
dataset_path = "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz"

if os.path.exists(dataset_path):
    print("📦 Extraindo dataset em inglês...")
    !tar -xzf "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz" -C data/generated/

    # Verificar extração
    print("\n📁 Conteúdo do dataset:")
    !ls -la data/generated/Dataset-Unificado/

    # Verificar metadados e idioma
    print("\n📋 Verificando estrutura do dataset:")
    !head -n 1 data/generated/Dataset-Unificado/metadata.csv

    print("\n🌍 Verificando idioma do dataset:")
    # Verificar coluna locale (posição 9 no CSV)
    !cut -d',' -f9 data/generated/Dataset-Unificado/metadata.csv | head -n 5 | tail -n 4

    print("\n📊 Estatísticas do dataset:")
    total_lines = !wc -l data/generated/Dataset-Unificado/metadata.csv
    print(f"Total de amostras: {total_lines[0].split()[0]} linhas")

    # Verificar algumas amostras de texto
    print("\n📝 Amostras de texto (verificando inglês):")
    !cut -d',' -f18 data/generated/Dataset-Unificado/metadata.csv | head -n 4 | tail -n 3

    print("\n✅ Dataset em inglês restaurado com sucesso!")
else:
    print(f"❌ Dataset não encontrado em: {dataset_path}")
    print("   Verifique se o arquivo existe no Google Drive")


In [ ]:
# 🔍 VERIFICAÇÃO COMPLETA E ROBUSTA DO SISTEMA
import os
import sys

print("🔍 Verificação completa do sistema:")
print(f"📁 Diretório atual: {os.getcwd()}")

# Garantir que o diretório atual está no PYTHONPATH e criar logs/
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"📝 Adicionado ao PYTHONPATH: {current_dir}")

# Criar diretórios essenciais se não existirem
essential_dirs = ['logs', 'outputs', 'samples', 'checkpoints']
for dir_name in essential_dirs:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
        print(f"📂 Criado diretório: {dir_name}")

# Verificar diretório correto (não aninhado)
if os.getcwd().endswith('/ValeTTS/ValeTTS'):
    print("🔧 Detectado diretório aninhado, corrigindo...")
    os.chdir('/content/ValeTTS')
    # Reconfigurar PYTHONPATH
    current_dir = os.getcwd()
    if current_dir not in sys.path:
        sys.path.insert(0, current_dir)

# Verificar estrutura essencial (INGLÊS)
essential_files = [
    'scripts/train_vits2.py',
    'configs/training/vits2_english_dataset.yaml',  # INGLÊS
    'valetts/__init__.py',
    'valetts/models/__init__.py',
    'valetts/models/vits2/__init__.py',
    'valetts/data/preprocessing/text_en.py'  # INGLÊS
]

print("\n📋 Verificando arquivos essenciais (INGLÊS):")
all_files_ok = True
for file in essential_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - FALTANDO")
        all_files_ok = False

# Verificar diretórios críticos
critical_dirs = [
    'valetts/models',
    'valetts/models/vits2',
    'valetts/data/preprocessing',
    'configs/training'
]

print("\n📂 Verificando diretórios críticos:")
for dir_path in critical_dirs:
    if os.path.isdir(dir_path):
        file_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
        print(f"✅ {dir_path} ({file_count} arquivos)")
    else:
        print(f"❌ {dir_path} - FALTANDO")
        all_files_ok = False

# Teste de import progressivo
print("\n🧪 Testando imports progressivos:")

try:
    import valetts
    print("✅ 1/4: valetts importado")

    from valetts.models import VITS2
    print("✅ 2/4: valetts.models.VITS2 importado")

    from valetts.data.preprocessing.text_en import EnglishTextPreprocessor
    print("✅ 3/4: EnglishTextPreprocessor importado")

    from valetts.training.trainers.vits2 import VITS2Trainer
    print("✅ 4/4: VITS2Trainer importado")

    print("🎉 TODOS OS IMPORTS FUNCIONANDO!")

except ImportError as e:
    print(f"❌ Erro no import: {e}")
    print("\n🔧 Tentando diagnóstico...")

    # Diagnóstico detalhado
    print(f"   PYTHONPATH: {sys.path[:3]}")
    print(f"   Diretório existe: {os.path.exists('valetts')}")
    print(f"   É diretório: {os.path.isdir('valetts')}")

    if os.path.exists('valetts'):
        valetts_contents = os.listdir('valetts')
        print(f"   Conteúdo valetts: {valetts_contents}")

        if 'models' in valetts_contents:
            models_contents = os.listdir('valetts/models')
            print(f"   Conteúdo models: {models_contents}")

# Testar script de treinamento SEM executar (apenas verificar sintaxe)
if os.path.exists('scripts/train_vits2.py'):
    print("\n🧪 Verificando sintaxe do script de treinamento...")
    try:
        with open('scripts/train_vits2.py', 'r') as f:
            script_content = f.read()

        # Compilar para verificar sintaxe
        compile(script_content, 'scripts/train_vits2.py', 'exec')
        print("✅ Script de treinamento tem sintaxe válida")

        # Verificar apenas o help (mais seguro)
        print("\n📖 Testando help do script...")
        !python scripts/train_vits2.py --help 2>&1 | head -10

    except SyntaxError as e:
        print(f"❌ Erro de sintaxe no script: {e}")
    except Exception as e:
        print(f"⚠️  Erro ao verificar script: {e}")
        print("   Script pode ter dependências não resolvidas, mas isso é normal antes da configuração completa.")

print(f"\n{'='*50}")
if all_files_ok:
    print("✅ SISTEMA ESTÁ PRONTO PARA INGLÊS!")
    print("   Todos os arquivos essenciais estão presentes.")
    print("   Pode prosseguir para o treinamento.")
else:
    print("⚠️  ALGUNS ARQUIVOS ESTÃO FALTANDO!")
    print("   Verifique se o git clone foi executado corretamente.")
    print("   Pode ser necessário executar novamente a clonagem do repositório.")
print(f"{'='*50}")


In [ ]:
# 🧪 TESTE FINAL E DEFINITIVO DE IMPORTS

import os
import sys

print("🧪 TESTE FINAL DE IMPORTS APÓS TODAS AS CORREÇÕES")
print("=" * 60)

# Garantir diretório e PYTHONPATH corretos uma última vez
current_dir = os.getcwd()
print(f"📁 Diretório atual: {current_dir}")

# Forçar limpeza e reconfiguração do PYTHONPATH
paths_to_remove = [p for p in sys.path if 'ValeTTS' in p]
for path in paths_to_remove:
    sys.path.remove(path)

# Adicionar apenas o caminho correto
if not current_dir.endswith('/ValeTTS'):
    if os.path.exists('/content/ValeTTS'):
        os.chdir('/content/ValeTTS')
        current_dir = os.getcwd()

sys.path.insert(0, current_dir)
print(f"🐍 PYTHONPATH limpo e reconfigurado: {current_dir}")

# Teste progressivo e definitivo
test_results = []

try:
    print("\n1️⃣ Testando import básico...")
    import valetts
    print(f"   ✅ valetts importado: {valetts.__file__}")
    test_results.append("✅ valetts")
except Exception as e:
    print(f"   ❌ valetts falhou: {e}")
    test_results.append("❌ valetts")

try:
    print("\n2️⃣ Testando models...")
    from valetts.models import VITS2
    print(f"   ✅ VITS2 importado: {VITS2}")
    test_results.append("✅ VITS2")
except Exception as e:
    print(f"   ❌ VITS2 falhou: {e}")
    test_results.append("❌ VITS2")

try:
    print("\n3️⃣ Testando EnglishTextPreprocessor...")
    from valetts.data.preprocessing.text_en import EnglishTextPreprocessor
    print(f"   ✅ EnglishTextPreprocessor importado: {EnglishTextPreprocessor}")
    test_results.append("✅ EnglishTextPreprocessor")
except Exception as e:
    print(f"   ❌ EnglishTextPreprocessor falhou: {e}")
    test_results.append("❌ EnglishTextPreprocessor")

try:
    print("\n4️⃣ Testando VITS2Trainer...")
    from valetts.training.trainers.vits2 import VITS2Trainer
    print(f"   ✅ VITS2Trainer importado: {VITS2Trainer}")
    test_results.append("✅ VITS2Trainer")
except Exception as e:
    print(f"   ❌ VITS2Trainer falhou: {e}")
    test_results.append("❌ VITS2Trainer")

# Teste do script de treinamento
try:
    print("\n5️⃣ Testando sintaxe do script de treinamento...")
    with open('scripts/train_vits2.py', 'r') as f:
        script_content = f.read()
    compile(script_content, 'scripts/train_vits2.py', 'exec')
    test_results.append("✅ Script válido")
    print("   ✅ Script tem sintaxe válida")
except Exception as e:
    print(f"   ❌ Script inválido: {e}")
    test_results.append("❌ Script inválido")

# Resultado final
print("\n" + "=" * 60)
print("📊 RESULTADO FINAL DOS TESTES:")
for result in test_results:
    print(f"   {result}")

success_count = len([r for r in test_results if r.startswith("✅")])
total_tests = len(test_results)

if success_count == total_tests:
    print(f"\n🎉 PERFEITO! {success_count}/{total_tests} TESTES PASSARAM!")
    print("   ✅ Sistema completamente funcional!")
    print("   ✅ Pronto para treinamento em inglês!")
    print("   ✅ Pode prosseguir para as próximas células!")
else:
    print(f"\n⚠️  {success_count}/{total_tests} testes passaram.")
    print("   🔧 Alguns problemas ainda existem.")
    print("   📝 Verifique os erros acima para diagnóstico.")

print("=" * 60)


In [ ]:
# Verificar e mostrar configuração que será usada (INGLÊS)
import yaml
import os

config_file = 'configs/training/vits2_english_dataset.yaml'

# Verificar se arquivo existe
if not os.path.exists(config_file):
    print(f"❌ Arquivo de configuração não encontrado: {config_file}")
    print("📁 Arquivos disponíveis em configs/training/:")
    !ls -la configs/training/ 2>/dev/null || echo "Diretório não encontrado"
else:
    # Carregar configuração
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)

    print("📋 Configuração do Treinamento:")

    # Função auxiliar para acessar chaves de forma segura
    def safe_get(dictionary, keys, default="N/A"):
        try:
            for key in keys:
                dictionary = dictionary[key]
            return dictionary
        except (KeyError, TypeError):
            return default

    # Mostrar configurações principais
    print(f"   • Dataset Dir: {safe_get(config, ['data', 'data_dir'])}")
    print(f"   • Metadata: {safe_get(config, ['data', 'metadata_file'])}")
    print(f"   • Speakers: {safe_get(config, ['model', 'n_speakers'])}")
    print(f"   • Epochs: {safe_get(config, ['training', 'max_epochs'])}")
    print(f"   • Batch Size: {safe_get(config, ['training', 'batch_size'])}")
    print(f"   • Learning Rate: {safe_get(config, ['training', 'learning_rate'])}")
    print(f"   • Mixed Precision: {safe_get(config, ['training', 'use_amp'])}")
    print(f"   • Sample Rate: {safe_get(config, ['data', 'sample_rate'])}")
    print(f"   • Mel Channels: {safe_get(config, ['data', 'n_mels'])}")

    # Monitor LLM
    if 'llm_monitor' in config and config['llm_monitor'].get('enabled', False):
        print(f"   • Monitor LLM: ✅ Ativado")
        print(f"     - Modelo: {safe_get(config, ['llm_monitor', 'model'])}")
        print(f"     - Monitor a cada: {safe_get(config, ['llm_monitor', 'monitor_every_epochs'])} épocas")
        print(f"     - Provider: {safe_get(config, ['llm_monitor', 'provider'])}")
    else:
        print(f"   • Monitor LLM: ❌ Desabilitado")

    print("\n🎯 Configuração carregada com sucesso!")


In [ ]:
# 🎙️ EXECUTAR TREINAMENTO VITS2 COM CONFIGURAÇÃO SELECIONADA

import os
import subprocess
import sys

# Verificar se a configuração foi definida
try:
    CONFIG_FILE
except NameError:
    print("❌ ERRO: Execute primeiro a célula de seleção de modo!")
    print("   Volte à primeira célula e defina DEBUG_MODE = True ou False")
    sys.exit(1)

# Verificar se o arquivo de configuração existe
if not os.path.exists(CONFIG_FILE):
    print(f"❌ ERRO: Arquivo de configuração não encontrado: {CONFIG_FILE}")
    print("   Verifique se o repositório foi clonado corretamente")
    sys.exit(1)

print("="*60)
print("🚀 INICIANDO TREINAMENTO VITS2 PARA INGLÊS...")
print(f"📁 Configuração: {CONFIG_FILE}")

if DEBUG_MODE:
    print("🎯 Modo: DEBUG (100 amostras, 3 épocas)")
else:
    print("🎯 Modo: PRODUCTION (22K+ amostras, 200 épocas)")

print("🌍 Dataset: Dataset-Unificado (inglês)")
print("🔧 Monitor LLM: Ativado" if 'OPENROUTER_API_KEY' in os.environ else "🔧 Monitor LLM: Desabilitado")
print("🎤 Geração de amostras: Ativada (textos em inglês)")
print(f"🐍 Python path: {os.getcwd()}")

if DEBUG_MODE:
    print("\n🐛 MODO DEBUG ATIVADO:")
    print("   • Apenas 100 amostras (90 train, 10 val)")
    print("   • 3 épocas para teste rápido")
    print("   • Batch size 2 para acelerar")
    print("   • Modelo reduzido para debug")
    print("   • Ideal para testar funcionamento")

print("="*60)

# Executar treinamento
cmd = [sys.executable, "scripts/train_vits2.py", "--config", CONFIG_FILE]

print(f"🎯 Comando: {' '.join(cmd)}")
print("\n🚀 Iniciando execução...\n")

# Executar com output em tempo real
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Mostrar output em tempo real
for line in process.stdout:
    print(line, end='')

# Aguardar conclusão
process.wait()

if process.returncode == 0:
    print("\n✅ TREINAMENTO CONCLUÍDO COM SUCESSO!")
    print("   🎵 Verifique as amostras geradas na próxima célula")
    print("   📊 Logs disponíveis no TensorBoard")
    print("   💾 Checkpoints salvos automaticamente")
else:
    print(f"\n❌ TREINAMENTO FALHOU (código: {process.returncode})")
    print("   📝 Verifique os logs acima para diagnóstico")
    print("   🔧 Se for modo debug, o erro pode ser configuração")
    print("   📋 Se for modo production, pode ser recursos insuficientes")


In [ ]:
# 🔧 INSTALAÇÃO ROBUSTA DO MÓDULO VALETTS

import subprocess
import sys
import os

def install_valetts_module():
    """Instala o módulo valetts em modo de desenvolvimento com verificações completas"""

    print("🔧 Verificando instalação do módulo valetts...")

    # Verificar se já está funcionando
    try:
        # Garantir PYTHONPATH
        current_dir = os.getcwd()
        if current_dir not in sys.path:
            sys.path.insert(0, current_dir)

        # Testar import completo
        import valetts
        from valetts.models import VITS2
        from valetts.data.preprocessing.text_en import EnglishTextPreprocessor
        print("✅ Módulo valetts já está funcionando completamente!")
        return True

    except ImportError as e:
        print(f"⚠️  Módulo valetts precisa ser instalado: {e}")

    # Verificar estrutura antes da instalação
    required_files = ['setup.py', 'pyproject.toml', 'valetts/__init__.py']
    missing_files = [f for f in required_files if not os.path.exists(f)]

    if missing_files:
        print(f"❌ Arquivos essenciais faltando: {missing_files}")
        print("   Verifique se o repositório foi clonado corretamente.")
        return False

    print("📦 Instalando módulo valetts...")

    try:
        # Método 1: pip install -e .
        print("   Tentativa 1: pip install -e .")
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-e", "."],
            cwd=os.getcwd(),
            capture_output=True,
            text=True,
            timeout=300  # 5 minutos timeout
        )

        if result.returncode == 0:
            print("✅ Instalação via pip concluída!")

            # Verificar se funcionou
            try:
                import valetts
                from valetts.models import VITS2
                print(f"✅ Módulo instalado e funcionando: {valetts.__file__}")
                return True
            except ImportError as e:
                print(f"⚠️  Pip install funcionou, mas import falhou: {e}")
        else:
            print(f"⚠️  Pip install falhou: {result.stderr}")

    except subprocess.TimeoutExpired:
        print("⏰ Timeout na instalação via pip")
    except Exception as e:
        print(f"❌ Erro na instalação via pip: {e}")

    # Método 2: PYTHONPATH manual
    print("\n   Tentativa 2: Configuração manual do PYTHONPATH")
    try:
        current_dir = os.getcwd()

        # Adicionar múltiplos caminhos
        paths_to_add = [
            current_dir,
            os.path.join(current_dir, 'valetts'),
        ]

        for path in paths_to_add:
            if path not in sys.path:
                sys.path.insert(0, path)

        # Configurar variável de ambiente
        os.environ['PYTHONPATH'] = ':'.join(paths_to_add + os.environ.get('PYTHONPATH', '').split(':'))

        # Testar import
        import valetts
        from valetts.models import VITS2
        print("✅ Configuração manual funcionou!")
        print(f"   PYTHONPATH: {sys.path[:3]}")
        return True

    except ImportError as e:
        print(f"❌ Configuração manual também falhou: {e}")

    return False

# Executar instalação com feedback detalhado
print("="*60)
success = install_valetts_module()

if success:
    print("\n🎉 MÓDULO VALETTS PRONTO!")
    print("   ✅ Imports funcionando")
    print("   ✅ Sistema pronto para treinamento")
    print("   ✅ Pode prosseguir para as próximas células")
else:
    print("\n🚨 MÓDULO VALETTS COM PROBLEMAS!")
    print("   ❌ Instalação falhou")
    print("   🔧 Soluções possíveis:")
    print("      1. Execute: !pip install -e . --force-reinstall")
    print("      2. Verifique se todos os arquivos foram clonados")
    print("      3. Reinicie o runtime e tente novamente")
    print("      4. Verifique se não há conflitos de versões")

print("="*60)


In [ ]:
# Carregar TensorBoard para monitoramento
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard/ --port 6006


In [ ]:
# Verificar progresso do treinamento
print("📊 Status do Treinamento:")
!ls -la logs/

print("\n📁 Checkpoints:")
!find logs/ -name "*.ckpt" | tail -5

print("\n📈 Logs mais recentes:")
!tail -10 logs/training.log 2>/dev/null || echo "Log não encontrado ainda"


In [ ]:
# Backup de modelos para Google Drive
import os
from shutil import copytree, ignore_patterns

backup_dir = "/content/drive/MyDrive/ValeTTS-Colab/training_backup"

if os.path.exists("logs/"):
    print("💾 Fazendo backup dos logs e checkpoints...")
    !mkdir -p "{backup_dir}"
    !cp -r logs/ "{backup_dir}/" 2>/dev/null || echo "Backup parcial realizado"
    print(f"✅ Backup salvo em: {backup_dir}")
else:
    print("ℹ️  Nenhum log encontrado para backup")


In [ ]:
# 🎵 EXPLORAR AMOSTRAS DE ÁUDIO GERADAS DURANTE O TREINAMENTO

import os
import glob
import json
from IPython.display import Audio, display, HTML
from pathlib import Path

def explore_generated_samples():
    """Explora e lista amostras de áudio geradas durante o treinamento."""

    samples_dir = Path("samples")

    if not samples_dir.exists():
        print("⚠️  Diretório 'samples' não encontrado.")
        print("   As amostras são geradas automaticamente durante o treinamento.")
        print("   Execute o treinamento primeiro para ver amostras aqui.")
        return

    # Listar épocas disponíveis
    epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])

    if not epoch_dirs:
        print("📂 Diretório samples existe, mas não há épocas com amostras ainda.")
        print("   As amostras serão geradas conforme o treinamento progride.")
        return

    print(f"🎵 Encontradas amostras de {len(epoch_dirs)} épocas!")
    print("\n📁 Épocas disponíveis:")

    for epoch_dir in epoch_dirs[-5:]:  # Mostrar últimas 5 épocas
        epoch_num = epoch_dir.name.split("_")[1]
        audio_files = list(epoch_dir.glob("*.wav"))
        print(f"   📊 Época {epoch_num}: {len(audio_files)} amostras")

    # Mostrar detalhes da época mais recente
    if epoch_dirs:
        latest_epoch = epoch_dirs[-1]
        epoch_num = latest_epoch.name.split("_")[1]

        print(f"\n🎯 ÉPOCA MAIS RECENTE: {epoch_num}")
        print(f"📁 Diretório: {latest_epoch}")

        # Verificar resumo da época
        summary_file = latest_epoch / "samples_summary.json"
        if summary_file.exists():
            with open(summary_file, 'r') as f:
                summary = json.load(f)

            print(f"📋 Resumo:")
            print(f"   🕐 Timestamp: {summary.get('timestamp', 'N/A')}")
            print(f"   🎤 Total de amostras: {summary.get('total_samples', 0)}")
            print(f"   🌍 Idioma: {summary.get('language', 'N/A')}")
            print(f"   👥 Speakers usados: {summary.get('speakers_used', 0)}")

        # Listar arquivos de áudio
        audio_files = sorted(latest_epoch.glob("*.wav"))
        if audio_files:
            print(f"\n🎶 Arquivos de áudio ({len(audio_files)}):")
            for audio_file in audio_files:
                print(f"   🔊 {audio_file.name}")

        return latest_epoch, audio_files

    return None, []

# Executar exploração
latest_epoch, audio_files = explore_generated_samples()


In [ ]:
# 🎧 REPRODUZIR AMOSTRAS DE ÁUDIO NO COLAB

def play_audio_samples(epoch_dir=None, max_samples=5):
    """Reproduz amostras de áudio no Colab com interface interativa."""

    if epoch_dir is None:
        # Usar época mais recente
        samples_dir = Path("samples")
        if not samples_dir.exists():
            print("⚠️  Nenhuma amostra encontrada. Execute o treinamento primeiro.")
            return

        epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])
        if not epoch_dirs:
            print("⚠️  Nenhuma época com amostras encontrada.")
            return

        epoch_dir = epoch_dirs[-1]

    epoch_num = epoch_dir.name.split("_")[1]
    print(f"🎵 REPRODUZINDO AMOSTRAS DA ÉPOCA {epoch_num}")
    print("=" * 50)

    # Buscar arquivos de áudio
    audio_files = sorted(epoch_dir.glob("*.wav"))

    if not audio_files:
        print("⚠️  Nenhum arquivo de áudio encontrado nesta época.")
        return

    # Carregar resumo se disponível
    summary_file = epoch_dir / "samples_summary.json"
    sample_metadata = {}

    if summary_file.exists():
        with open(summary_file, 'r') as f:
            summary = json.load(f)

        # Criar dicionário de metadados por arquivo
        for sample in summary.get('samples', []):
            audio_path = Path(sample['audio_path'])
            sample_metadata[audio_path.name] = sample

    # Reproduzir amostras
    played_count = 0
    for audio_file in audio_files[:max_samples]:
        if played_count >= max_samples:
            break

        print(f"\n🔊 {audio_file.name}")

        # Mostrar metadados se disponíveis
        if audio_file.name in sample_metadata:
            metadata = sample_metadata[audio_file.name]
            print(f"   📝 Texto: {metadata['text']}")
            print(f"   🎤 Speaker: {metadata['speaker_name']} ({metadata['speaker_id']})")
            if 'metadata' in metadata and 'audio_length_seconds' in metadata['metadata']:
                duration = metadata['metadata']['audio_length_seconds']
                print(f"   ⏱️  Duração: {duration:.2f}s")

        # Reproduzir áudio
        try:
            audio_widget = Audio(str(audio_file), autoplay=False)
            display(audio_widget)
            played_count += 1

        except Exception as e:
            print(f"   ❌ Erro ao carregar áudio: {e}")

    if len(audio_files) > max_samples:
        remaining = len(audio_files) - max_samples
        print(f"\n📊 Mostrando {max_samples} de {len(audio_files)} amostras.")
        print(f"   (+{remaining} amostras adicionais disponíveis)")

    print(f"\n✅ Reprodução concluída! {played_count} amostras carregadas.")

# Reproduzir amostras se disponíveis
if 'latest_epoch' in locals() and latest_epoch:
    play_audio_samples(latest_epoch, max_samples=3)
else:
    print("🎯 Execute a célula anterior primeiro para encontrar amostras disponíveis.")
    print("   Ou rode esta célula após o treinamento ter gerado algumas amostras.")


In [ ]:
# 💾 BACKUP DE AMOSTRAS DE ÁUDIO PARA GOOGLE DRIVE

import shutil
from pathlib import Path

def backup_audio_samples_to_drive():
    """Faz backup das amostras geradas para o Google Drive."""

    samples_dir = Path("samples")
    backup_base = Path("/content/drive/MyDrive/ValeTTS-Colab")
    backup_samples_dir = backup_base / "audio_samples"

    if not samples_dir.exists():
        print("⚠️  Diretório 'samples' não encontrado.")
        print("   Execute o treinamento primeiro para gerar amostras.")
        return False

    # Criar diretório de backup
    backup_samples_dir.mkdir(parents=True, exist_ok=True)

    # Contar épocas e amostras
    epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])

    if not epoch_dirs:
        print("📂 Diretório samples existe, mas não há épocas para fazer backup.")
        return False

    print(f"💾 Fazendo backup de {len(epoch_dirs)} épocas para Google Drive...")
    print(f"📁 Destino: {backup_samples_dir}")

    total_files = 0
    total_size = 0

    for epoch_dir in epoch_dirs:
        epoch_backup_dir = backup_samples_dir / epoch_dir.name

        try:
            # Copiar época inteira
            if epoch_backup_dir.exists():
                shutil.rmtree(epoch_backup_dir)

            shutil.copytree(epoch_dir, epoch_backup_dir)

            # Contar arquivos
            audio_files = list(epoch_backup_dir.glob("*.wav"))
            epoch_size = sum(f.stat().st_size for f in epoch_backup_dir.rglob("*") if f.is_file())

            total_files += len(audio_files)
            total_size += epoch_size

            print(f"   ✅ {epoch_dir.name}: {len(audio_files)} amostras ({epoch_size/1024/1024:.1f} MB)")

        except Exception as e:
            print(f"   ❌ Erro no backup de {epoch_dir.name}: {e}")

    print(f"\n📊 BACKUP CONCLUÍDO:")
    print(f"   🎵 Total de amostras: {total_files}")
    print(f"   💾 Tamanho total: {total_size/1024/1024:.1f} MB")
    print(f"   📁 Salvo em: {backup_samples_dir}")

    # Criar arquivo de índice
    index_file = backup_samples_dir / "index.txt"
    with open(index_file, 'w') as f:
        f.write(f"ValeTTS Audio Samples Backup\\n")
        f.write(f"Generated: {Path.cwd()}\\n")
        f.write(f"Total epochs: {len(epoch_dirs)}\\n")
        f.write(f"Total samples: {total_files}\\n")
        f.write(f"Total size: {total_size/1024/1024:.1f} MB\\n\\n")

        for epoch_dir in epoch_dirs:
            epoch_backup_dir = backup_samples_dir / epoch_dir.name
            if epoch_backup_dir.exists():
                audio_files = list(epoch_backup_dir.glob("*.wav"))
                f.write(f"{epoch_dir.name}: {len(audio_files)} samples\\n")

    print(f"   📋 Índice salvo: {index_file}")

    return True

# Executar backup se há amostras disponíveis
if Path("samples").exists():
    backup_success = backup_audio_samples_to_drive()

    if backup_success:
        print("\\n🎉 BACKUP CONCLUÍDO COM SUCESSO!")
        print("   Suas amostras de áudio estão seguras no Google Drive.")
        print("   Você pode baixá-las ou acessá-las diretamente do Drive.")
    else:
        print("\\n⚠️  Backup não foi necessário ou falhou.")
else:
    print("📝 Backup será executado automaticamente quando houver amostras disponíveis.")
    print("   Execute o treinamento primeiro para gerar amostras de áudio.")
